In [1]:
!pip install datasets scikit-learn python-dotenv


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from huggingface_hub import login
from dotenv import dotenv_values

login(token=dotenv_values('.env')['HUGGING_FACE_TOKEN'])

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to C:\Users\hakatashi\.cache\huggingface\token
Login successful


In [3]:
from datasets import load_dataset

dataset = load_dataset("hakatashi/hakatashi-pixiv-bookmark-deepdanbooru-private")

In [4]:
dataset

DatasetDict({
    test: Dataset({
        features: ['key', 'tag_probs', 'class'],
        num_rows: 59707
    })
    train: Dataset({
        features: ['key', 'tag_probs', 'class'],
        num_rows: 179121
    })
    validation: Dataset({
        features: ['key', 'tag_probs', 'class'],
        num_rows: 59708
    })
})

In [5]:
dataset['test'].features

{'key': Value(dtype='string', id=None),
 'tag_probs': Sequence(feature=Value(dtype='float32', id=None), length=-1, id=None),
 'class': ClassLabel(names=['not_bookmarked', 'bookmarked_public', 'bookmarked_private'], id=None)}

In [6]:
np_dataset = dataset.with_format(type='numpy')

In [7]:
train_data = np_dataset['train'][:]
test_data = np_dataset['test'][:]
validation_data = np_dataset['validation'][:]

In [8]:
%%time

from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(max_depth=10)
clf.fit(train_data['tag_probs'], train_data['class'])

CPU times: total: 26min 4s
Wall time: 26min 23s


DecisionTreeClassifier(max_depth=10)

In [9]:
from joblib import dump
dump(clf, 'sklearn-multiclass-decision-tree-depth-10.joblib')

['sklearn-multiclass-decision-tree-depth-10.joblib']

In [10]:
clf.score(test_data['tag_probs'], test_data['class'])

0.849582126048872

In [11]:
predicted_class = clf.predict(test_data['tag_probs'])

In [12]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
print('confusion_matrix:')
print(confusion_matrix(test_data["class"], predicted_class))
print(f'accuracy_score: {accuracy_score(test_data["class"], predicted_class)}')
print(f'precision_score: {precision_score(test_data["class"], predicted_class, average="macro")}')
print(f'recall_score: {recall_score(test_data["class"], predicted_class, average="macro")}')
print(f'f1_score: {f1_score(test_data["class"], predicted_class, average="macro")}')

confusion_matrix:
[[37332  2652   714]
 [ 3236 11027   256]
 [ 1374   749  2367]]
accuracy_score: 0.849582126048872
precision_score: 0.7878946170408506
recall_score: 0.7346507644339774
f1_score: 0.7567298609545833


In [13]:
len(list(filter(lambda tup: tup[0] == 2 and tup[1] != 2, zip(predicted_class, test_data['class'], test_data['key']))))

970

In [14]:
!pip install matplotlib


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
from urllib import request
import json

res = request.urlopen('https://raw.githubusercontent.com/RF5/danbooru-pretrained/master/config/class_names_6000.json')
class_names = json.load(res)

In [ ]:
from sklearn import tree

print(tree.export_text(clf, feature_names=class_names, class_names=dataset['test'].features['class'].names, decimals=5, max_depth=6))